In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
# 读取mnist数据
mnist = input_data.read_data_sets("/Users/xxx/work5/tensorflow/tensorflow-tutorial/Deep_Learning_with_TensorFlow/datasets/MNIST_data", dtype=tf.uint8, one_hot=True)
images = mnist.train.images
labels = mnist.train.labels

pixels = images.shape[1]
num_examples = mnist.train.num_examples
print images
print labels
print images.shape
print num_examples

Extracting /Users/xxx/work5/tensorflow/tensorflow-tutorial/Deep_Learning_with_TensorFlow/datasets/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/tensorflow-tutorial/Deep_Learning_with_TensorFlow/datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/tensorflow-tutorial/Deep_Learning_with_TensorFlow/datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/tensorflow-tutorial/Deep_Learning_with_TensorFlow/datasets/MNIST_data/t10k-labels-idx1-ubyte.gz
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
[[ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
(55000, 784)
55000


In [4]:
# 将输入转化成TFRecord格式并保存

# 定义函数转化变量类型
# 生成整数型的属性
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
# 生成字符串型的属性
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# 输出TFRecord文件的地址
filename = "./output.tfrecords"
# 创建一个writer来写TFRecord文件
writer = tf.python_io.TFRecordWriter(filename)

for index in range(num_examples):
    # 将图像矩阵转化成一个字符串
    image_raw = images[index].tostring()
    
    # 将一个样例转化为Example Protocol Buffer，并将所有的信息写入这个数据结构
    example = tf.train.Example(features=tf.train.Features(feature={
                'pixels': _int64_feature(pixels),
                'label': _int64_feature(np.argmax(labels[index])),
                'image_raw': _bytes_feature(image_raw)
            }))
    
    # 将一个Example写入TFRecord文件
    writer.write(example.SerializeToString())


writer.close()

print "save success"

save success


In [ ]:
# tf 读取tfrecord数据

# 创建一个reader来读取TFRecord文件中的样例
reader = tf.TFRecordReader()

# 创建一个队列来维护输入文件的列表
filename_queue = tf.train.string_input_producer(['./output.tfrecords'])

# 从文件中国年读取一个样例
# read_up_to函数一次性读取多个样例
_, serialized_example = reader.read(filename_queue)

# 解析读入的一个样例
# parse_example解析多个样例
features = tf.parse_single_example(
    serialized_example,
    # Tensorflow 提供了两种不同的属性解析方法
    # tf.FixedLenFeature，解析的结果为一个Tensor
    # tf.VarLenFeature，解析的结果为SparseTensor，用于处理稀疏矩阵
    # 解析数据的格式需要和上面写入的数据格式一致
    features={
        'pixels': tf.FixedLenFeature([], tf.int64),
        'label': tf.FixedLenFeature([], tf.int64),
        'image_raw': tf.FixedLenFeature([], tf.string)
    }
)

# tf.decode_raw将字符串解析成图像对应的像素数组
images = tf.decode_raw(features['image_raw'], tf.uint8)
labels = tf.cast(features['label'], tf.int32)
pixels = tf.cast(features['pixels'], tf.int32)

with tf.Session() as sess:
    # 启动多线程处理输入数据
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    print "threads: ", threads
    
    # 每次运行可以读取TFRecord文件中的一个样例
    # 当所有样例都读完之后，在此样例中程序会在重头读取
    for i in range(10):
        image, label, pixel = sess.run([images, labels, pixels])
        print image

print "end"